In [1]:
import requests
from lxml import etree
import json
import jieba
from jieba.analyse  import extract_tags
import plotly.graph_objects as go


In [2]:
requests = requests.Session()

In [3]:
h = {
            'Host': 'www.dcard.tw',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'zh-TW,zh;q=0.8,en-US;q=0.5,en;q=0.3',
            'Accept-Encoding': 'gzip, deflate',
            'DNT': '1',
            'Connection': 'keep-alive',
            'Cookie': '__cfduid=d043f4381bec87c48df7b199ba78c36921592574408; dcsrd=wzYupjK9-4rPWoZK-kgcIZTI; dcsrd.sig=aMutTs7K3XR1RQj7l7jZ4zNC6Ug; __asc=698b752b172ccd66e60423e1f66; __auc=698b752b172ccd66e60423e1f66; _ga=GA1.1.1003447311.1592574505; _ga_C3J49QFLW7=GS1.1.1592574504.1.1.1592574594.0',
            'Upgrade-Insecure-Requests': '1'
}


In [4]:
star_sign = {
'羊':0,
'牛':0,
'雙子':0,
'蟹':0,
'獅':0,
'處女':0,
'天秤': 0,
'蠍': 0,
'射手':0,
'羯':0,
'水瓶':0,
'魚':0}

In [5]:
p = requests.get(
    'https://www.dcard.tw/f/horoscopes',
    headers = h
)
HTML = etree.HTML(p.text)
Title = HTML.xpath('//a[@class="sc-1v1d5rx-3 kPUUNB"]//text()')



Total_UID = HTML.xpath('//a[@class="sc-1v1d5rx-3 kPUUNB"]//@href')
for U in Total_UID:
    UID = U.split('/')[-1]

i = 0
while i<11:
    p1 = requests.get(
        'https://www.dcard.tw/service/api/v2/forums/horoscopes/posts',
        headers = h,
        params = {
            'limit':'30',
            'before':UID
        }
    )
    a = json.loads(p1.text)
    for b in a:
        Title.append(b['title'])
    UID = b['id']
    i += 1

In [7]:
jieba.set_dictionary('drive/My Drive/dict.txt.big.txt')
for T in Title:
    Title_extract = jieba.lcut(T, cut_all=False)
    for TE in Title_extract:
        for S in star_sign:
            if S in TE:
                star_sign[S] += 1

Building prefix dict from /content/drive/My Drive/dict.txt.big.txt ...
Dumping model to file cache /tmp/jieba.u4cb55ab715802e96bd1e565a968437ad.cache
Loading model cost 1.590 seconds.
Prefix dict has been built successfully.


In [8]:
star_sign["牡羊座"] = star_sign.pop("羊")
star_sign["金牛座"] = star_sign.pop("牛")
star_sign["雙子座"] = star_sign.pop("雙子")
star_sign["巨蟹座"] = star_sign.pop("蟹")
star_sign["獅子座"] = star_sign.pop("獅")
star_sign["處女座"] = star_sign.pop("處女")
star_sign["天秤座"] = star_sign.pop("天秤")
star_sign["天蠍座"] = star_sign.pop("蠍")
star_sign["射手座"] = star_sign.pop("射手")
star_sign["水瓶座"] = star_sign.pop("水瓶")
star_sign["魔羯座"] = star_sign.pop("羯")
star_sign["雙魚座"] = star_sign.pop("魚")

In [9]:
star_sign

{'天秤座': 17,
 '天蠍座': 42,
 '射手座': 18,
 '巨蟹座': 49,
 '水瓶座': 23,
 '牡羊座': 24,
 '獅子座': 37,
 '處女座': 29,
 '金牛座': 28,
 '雙子座': 17,
 '雙魚座': 19,
 '魔羯座': 40}

In [10]:
Bar = go.Bar(x = list(star_sign.keys()), y = list(star_sign.values()),
             text = list(star_sign.values()), textposition = 'outside')
fig = go.Figure(Bar)
fig.update_layout(title = '星座討論熱度圖',
                  xaxis_title = '星座',
                  yaxis_title = '討論度')
fig.show()